In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Define dataset file paths
datasets = {
    "gas_price": "gas_price.csv",
    "rig_count": "rig_count.csv",
    "tam": "tam.csv",
    "capex": "capex.csv",
    "oil_price": "oil_price.csv",
    "education": "education.csv",
    "political_stability": "political_stability.csv",
    "infrastructure_index": "infrastructure_index.csv",
    "health_index": "health_index.csv",
    "gni": "gni.xlsx",
    "gdp": "gdp.xlsx",
    "population": "population.xlsx",
    "country_mapping": "country_table(in).csv"
}

def load_data(dataset_name):
    """Loads and processes the dataset."""
    path = datasets.get(dataset_name)
    if not path:
        print(f"Dataset '{dataset_name}' not found in provided paths.")
        return None

    try:
        if path.lower().endswith('.csv'):
            df = pd.read_csv(path)
        elif path.lower().endswith(('.xlsx', '.xls')):
            df = pd.read_excel(path)
        else:
            print(f"Unsupported file format for dataset '{dataset_name}'.")
            return None

        df["country_name"] = df["country_name"].str.strip().str.upper()
        df["year"] = df["year"].astype(int)
        return df
    except Exception as e:
        print(f"Error loading {dataset_name}: {e}")
        return None

# ---------- Model Building ----------
def build_nn_model(input_dim):
    """Builds and returns a neural network model."""
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# ---------- Training Country-Specific Models ----------
def train_model_for_country(dataset_name, country):
    df = load_data(dataset_name)
    if df is None:
        print(f"Skipping {country}: Unable to load data for {dataset_name}.")
        return None, None, None, None

    country_df = df[df["country_name"] == country]
    if country_df.empty:
        print(f"Skipping {country}: No data available for {dataset_name}.")
        return None, None, None, None

    # Determine target feature: first column not in country_name/year
    target_feature = [col for col in country_df.columns if col not in ["country_name", "year"]][0]
    scaler = MinMaxScaler(feature_range=(0, 1))

    country_df_sorted = country_df.sort_values("year")
    country_df_sorted.interpolate(method="linear", inplace=True)

    X = country_df_sorted.drop(columns=["country_name", "year"])
    y = country_df_sorted[[target_feature]]

    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(y)

    if len(X_scaled) < 5:
        print(f"Skipping {country}: Not enough data for training in {dataset_name}.")
        return None, None, None, None

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_scaled, test_size=0.2, random_state=42)
    model = build_nn_model(input_dim=X_train.shape[1])

    lr_scheduler = ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[lr_scheduler, early_stopping],
        verbose=2
    )

    return model, scaler, country_df_sorted, target_feature

# ---------- Training Year-Based Models ----------
def train_nn_model(df, target_feature):
    grouped_df = df.groupby("year").mean(numeric_only=True).reset_index()
    X = grouped_df[["year"]].values
    y = grouped_df[[target_feature]].values

    scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_scaled, test_size=0.2, random_state=42)
    model = build_nn_model(input_dim=1)

    lr_scheduler = ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=32,
        callbacks=[lr_scheduler, early_stopping],
        verbose=2
    )

    return model, scaler_X, scaler_y

# ---------- Prediction Functions ----------
def predict_future_by_country(model, scaler, df, target_feature, start_year, num_years, country):
    if model is None:
        return pd.DataFrame()

    future_predictions = []
    latest_data = df[df["year"] == df["year"].max()]
    latest_scaled = scaler.transform(
        latest_data.drop(columns=["country_name", "year"]))

    for i in range(num_years):
        predicted_scaled = model.predict(latest_scaled, verbose=0)
        predicted_value = scaler.inverse_transform(predicted_scaled)[0][0]
        trend_factor = np.random.uniform(0.9, 1.1)
        predicted_value *= trend_factor

        if target_feature == "rig_count":
            predicted_value = int(round(predicted_value))

        prediction_year = start_year + i
        future_predictions.append({
            "year": prediction_year,
            "country_name": country,
            target_feature: predicted_value
        })
        print(f"Predicted {target_feature} for {country} in {prediction_year}: {predicted_value}")

    return pd.DataFrame(future_predictions)

def predict_prices(model, scaler_X, scaler_y, start, end, countries, feature_name):
    predictions = []
    years = np.arange(start, end + 1).reshape(-1, 1)
    years_scaled = scaler_X.transform(years)
    preds_scaled = model.predict(years_scaled, verbose=0)
    preds = scaler_y.inverse_transform(preds_scaled)

    for country in countries:
        for year, price in zip(years.flatten(), preds.flatten()):
            predictions.append({
                "year": year,
                "country_name": country,
                feature_name: price
            })
    return pd.DataFrame(predictions)

# --------------------------------- Main Execution Flow -------------------------------------
valid_datasets = {"gas_price", "oil_price", "rig_count", "tam", "capex"}
dataset_names_input = input(
    "\nEnter what you want to predict (gas_price, oil_price, rig_count, tam, capex): ")
dataset_names = [
    name.strip() for name in dataset_names_input.split(',')
    if name.strip() in valid_datasets
]

for i, name in enumerate(datasets.keys(), start=1):
    print(f"{i}. {name}")

print("\n1. Gas Price Prediction: Uses Oil Price, Rig Count, Infrastructure Index, Capex.")
print("2. Rig Count Prediction: Uses Rig Count, Infrastructure Index, Capex, Political Stability.")
print("3. Oil Price Prediction: Uses Rig Count, GDP Growth, Infrastructure Index, Capex, Political Stability.")
print("4. Tam Prediction: Uses Capex, GDP, Population, Health Index, Education, Infrastructure Index.")
print("5. Capex Prediction: Uses Oil Price, Gas Price, Rig Count, GDP Growth, Political Stability, Infrastructure Index.")

selected_options = input(
    "\nEnter the numbers corresponding to the datasets you want to predict: "
).strip()

# Load country mapping to get available countries
gas_price_data = load_data("gas_price")
available_countries = sorted(
    gas_price_data["country_name"].unique()
) if gas_price_data is not None else []
print(
    "\nAvailable countries: ",
    ", ".join(available_countries) if available_countries else "None"
)

selected_countries_input = input(
    "Enter countries (comma-separated): "
)
selected_countries = [
    c.strip().upper() for c in selected_countries_input.split(',')
    if c.strip().upper() in available_countries
]

start_year = int(input("Enter the start year for predictions: "))
end_year = int(input("Enter the end year for predictions: "))

# Session ID and Company ID input
session_id = int(input("Enter session id: "))
company_id = int(input("Enter company id: "))

# Train global models for gas and oil prices
oil_price_data = load_data("oil_price")
gas_model, gas_scaler_X, gas_scaler_y = train_nn_model(
    gas_price_data, "gas_price"
)
oil_model, oil_scaler_X, oil_scaler_y = train_nn_model(
    oil_price_data, "oil_price"
)

# Create predictions skeleton
years = list(range(start_year, end_year + 1))
skeleton = pd.DataFrame([
    (year, country) for country in selected_countries for year in years
], columns=["year", "country_name"] )
final_predictions = skeleton.copy()

# Merge dataset predictions
for dataset in dataset_names:
    if dataset in ["gas_price", "oil_price"]:
        pred_df = predict_prices(
            gas_model if dataset=="gas_price" else oil_model,
            gas_scaler_X if dataset=="gas_price" else oil_scaler_X,
            gas_scaler_y if dataset=="gas_price" else oil_scaler_y,
            start_year, end_year, selected_countries, dataset
        )
        final_predictions = final_predictions.merge(
            pred_df, on=["year", "country_name"], how="left"
        )
    else:
        all_preds = []
        for country in selected_countries:
            model, scaler, df, target = train_model_for_country(dataset, country)
            if model:
                num_years = end_year - start_year + 1
                all_preds.append(
                    predict_future_by_country(
                        model, scaler, df, target,
                        start_year, num_years, country
                    )
                )
        if all_preds:
            final_predictions = final_predictions.merge(
                pd.concat(all_preds, ignore_index=True),
                on=["year", "country_name"], how="left"
            )

# Load and merge country mapping
country_mapping = pd.read_csv(datasets["country_mapping"])
country_mapping["country_name"] = country_mapping["country_name"].str.strip().str.upper()
final_predictions = final_predictions.merge(
    country_mapping[["country_name","country_id"]],
    on="country_name", how="left"
)

# Add session_id and company_id columns
final_predictions['session_id'] = session_id
final_predictions['company_id'] = company_id

# Reorder columns including session_id and company_id
cols = [
    "session_id",
    "company_id",
    "country_id",
    "country_name",
    "year"
] + [col for col in final_predictions.columns if col not in {"session_id","company_id","country_id","country_name","year"}]
final_predictions = final_predictions[cols]

# Save predictions
output_file = input(
    "\nEnter the file name to save predictions (e.g., predictions_output.csv): "
).strip()
if not output_file.lower().endswith('.csv'):
    output_file += '.csv'
final_predictions.to_csv(output_file, index=False)
print(f"\nPredictions have been successfully saved to '{output_file}'.")